In [3]:
import pandas as pd
import numpy as np
import folium
import json
from geopy.geocoders import Nominatim
import requests

In [5]:
address = "Taichung"
geolocater = Nominatim(user_agent='Bar_explorer')
location = geolocater.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}'.format(address,latitude,longitude))

The geograpical coordinate of Taichung are 24.163162, 120.6478282


In [72]:
url = 'https://en.wikipedia.org/wiki/District_(Taiwan)'
df = pd.read_html(url)[4]
df_taichung = df[df['City']=='Taichung']
df_taichung.drop(['Hànyǔ Pīnyīn','Taiwanese POJ','Hakka','Formosan or others','Type'],axis=1,inplace=True)
df_taichung.reset_index(drop=True,inplace=True)
df_taichung['Chinese']='台中市'+df_taichung['Chinese']
df_taichung.rename(columns={'Chinese':'縣市各區'},inplace=True)
df_taichung.head(10)

/Users/hank/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/hank/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Name,縣市各區,City
0,Central,台中市中區,Taichung
1,East,台中市東區,Taichung
2,South,台中市南區,Taichung
3,West,台中市西區,Taichung
4,North,台中市北區,Taichung
5,Beitun,台中市北屯區,Taichung
6,Xitun,台中市西屯區,Taichung
7,Nantun,台中市南屯區,Taichung
8,Taiping,台中市太平區,Taichung
9,Dali,台中市大里區,Taichung


In [70]:
url_2='http://www.astrocode.net/%E5%8F%B0%E7%81%A3%E5%90%84%E7%B8%A3%E5%B8%82%E5%9C%B0%E5%8D%80%E7%B6%93%E7%B7%AF%E5%BA%A6/'
df_2=pd.read_html(url_2)[0]
new_header=df_2.loc[0]
new_header=new_header.to_list()
df_2.columns = new_header
df_2=df_2[88:112]
df_2.reset_index(drop=True,inplace=True)
df_2.head(10)

,主要縣市,縣市各區,Longitude (經度),Latitude(緯度)
0,台中,台中市大安區,120E35’00”,24N21’00”
1,NaN,台中市大甲區,120E31’00”,24N21’00”
2,NaN,台中市外埔區,120E39’00”,24N20’00”
3,NaN,台中市后里區,120E42’00”,24N18’00”
4,NaN,台中市石崗區,120E46’00”,24N17’00”
5,NaN,台中市清水區,120E34’00”,24N16’00”
6,NaN,台中市梧棲區,120E31’00”,24N15’00”
7,NaN,台中市神崗區,120E39’00”,24N15’00”
8,NaN,台中市豐原區,120E43’00”,24N15’00”
9,NaN,台中市新社區,120E48’00”,24N15’00”


In [81]:
taichung_data = pd.merge(df_taichung,df_2,on='縣市各區')
taichung_data.head(5)

,Name,縣市各區,City,主要縣市,Longitude (經度),Latitude(緯度)
0,Beitun,台中市北屯區,Taichung,NaN,120E41’00”,24N10’00”
1,Xitun,台中市西屯區,Taichung,NaN,120E37’00”,24N11’00”
2,Nantun,台中市南屯區,Taichung,NaN,120E37’00”,24N08’00”
3,Taiping,台中市太平區,Taichung,NaN,120E42’00”,24N08’00”
4,Dali,台中市大里區,Taichung,NaN,120E40’00”,24N06’00”


In [82]:
taichung_data.drop('主要縣市',axis=1,inplace=True)
column = ['District','District in Chinese','City','Longitude','Latitude']
taichung_data.columns = column
taichung_data.head(5)

,District,District in Chinese,City,Longitude,Latitude
0,Beitun,台中市北屯區,Taichung,120E41’00”,24N10’00”
1,Xitun,台中市西屯區,Taichung,120E37’00”,24N11’00”
2,Nantun,台中市南屯區,Taichung,120E37’00”,24N08’00”
3,Taiping,台中市太平區,Taichung,120E42’00”,24N08’00”
4,Dali,台中市大里區,Taichung,120E40’00”,24N06’00”


In [83]:
taichung_data.dtypes

District               object
District in Chinese    object
City                   object
Longitude              object
Latitude               object
dtype: object

In [86]:
import re

In [142]:
def dms2dd(s):
    degree,direction,minute,n0,second,n1,n2=re.split('([\D’”])',s)
    dd = float(degree) + float(minute)/60 + float(second)/(60*60)
    if direction in ('S','W'):
        dd*=-1
    return dd

In [144]:
taichung_data['Longitude'] = taichung_data['Longitude'].apply(dms2dd)
taichung_data['Latitude'] = taichung_data['Latitude'].apply(dms2dd)
taichung_data.head(5)

,District,District in Chinese,City,Longitude,Latitude
0,Beitun,台中市北屯區,Taichung,120.683333,24.166667
1,Xitun,台中市西屯區,Taichung,120.616667,24.183333
2,Nantun,台中市南屯區,Taichung,120.616667,24.133333
3,Taiping,台中市太平區,Taichung,120.700000,24.133333
4,Dali,台中市大里區,Taichung,120.666667,24.100000


In [155]:
map_taichung=folium.Map(location=[latitude,longitude],zoom_start=11)
# add markers to map
for lat, lng, label in zip(taichung_data['Latitude'], taichung_data['Longitude'], taichung_data['District']):
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        [lat, lng],
        popup=label,).add_to(map_taichung)  
    
map_taichung

In [156]:
taichung_data

,District,District in Chinese,City,Longitude,Latitude
0,Beitun,台中市北屯區,Taichung,120.683333,24.166667
1,Xitun,台中市西屯區,Taichung,120.616667,24.183333
2,Nantun,台中市南屯區,Taichung,120.616667,24.133333
3,Taiping,台中市太平區,Taichung,120.700000,24.133333
4,Dali,台中市大里區,Taichung,120.666667,24.100000
5,Wufeng,台中市霧峰區,Taichung,120.683333,24.066667
6,Wuri,台中市烏日區,Taichung,120.616667,24.100000
7,Fengyuan,台中市豐原區,Taichung,120.716667,24.250000
8,Houli,台中市后里區,Taichung,120.700000,24.300000
9,Dongshi,台中市東勢區,Taichung,120.816667,24.250000
